<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Chat about a deal
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Chat_about_a_deal.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Chat+about+a+deal:+Error+short+description">Bug report</a>

**Tags:** #hubspot #chat #deals #last #discussion #conversation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-25 (Created: 2023-09-25)

**Description:** This notebook aims to create a Naas Chat Plugin that allows Sales to easily chat about any deal in HubSpot by simply passing its URL. The plugin will provide essential insights about the deal and its recent activities, conveniently displaying them in the input bar of your chat. This notebook will leverage the /command feature of the Naas Chat and a webhook to dynamically invoke the HubSpot API and retrieve updated values.

### Import libraries

In [15]:
import requests
import naas
import naas_drivers
from naas_drivers import hubspot, naas_chat_plugin
import pandas as pd
import os
from IPython.display import Markdown
pd.set_option('display.max_colwidth', None)

## Input

### Setup variables
**Mandatory**
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API. [Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `deal_url`: This variable stores the HubSpot deal URL.

**Optional**
- `body`: This variable stores the body to be send by the webhook.
- `deal_properties`: It represents the list of properties to retrieve from your deal
- `associations`: It represents the list of associations to get from your deal
- `plugin_name`: It represents the name of the plugin.

In [16]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
deal_url = "https://app.hubspot.com/contacts/2474088/record/0-3/15220482319/" # "https://app.hubspot.com/contacts/xxxxx/record/0-1/xxxxx"

# Optional
body = {}
deal_properties = ["hs_object_id", "dealname", 'amount']
associations = ["notes", "emails", "meetings", "communications"]
plugin_name = "Sales Agent"

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [17]:
# Parameters
if len(body) > 0:
    deal_url = body.get("deal_url") # If a URL is received through a webhook, the notebook will extract the deal URL from the body of the request.

## Model

### Set System prompt
System prompt to be used in the Naas Chat plugin.

In [18]:
system_prompt = '''Act as a Sales Assistant expert working in analyzing conversations and extracting action items.
Start presenting yourself and tell the user they can do the slash command /HubSpot_Get_deal_summary to query deal notes, meetings, calls. 
Once you have this information, you can start list action items based on MEDDIC sales techniques. 
Each new deal you get from that command should be presented as follows: 

MEDDIC Analysis: 

1/ Metric: The economic impact of the solution is...

2/ Economic Buyer: The person with the budget is...

3/ Decision Criteria: The formal criteria used to compare vendor's offering in terms of: 
- Capabilities is...
- Vendor info is...
- Return on investment is...  

4/ Decision Process: The events and timeline in the validation and approval process is...

5/ Identify Pain: The pain is... It impacts the following KPIs: 
- KPI 1: 
- KPI 2: 
- KPI 3: 

The compelling event attached is that...

The cost of doing nothing is...

6/ Champion: The key player who has the power and influence in driving the opportunity and who can sell on our behalf is...
'''

### Create a template message
The variables "[URL]" and "[ACTIVITIES]" will be replaced to be sent in the input bar of your Naas Chat.

In [19]:
message = f"""Here is the deal URL: [URL].
Here are all the activities:
[ACTIVITIES]
"""

### Get deal ID from URL

In [20]:
def get_contact_ID_from_URL(url):
    # Init
    uid = url
    
    # Check if URL is valid
    if not url.startswith("https://app.hubspot.com/contacts/"):
        raise BaseException("HubSpot URL Invalid! It must start by https://app.hubspot.com/contacts/")
    
    # Split URL to get ID
    if "/record/0-3/" in url:
        uid = url.split("/record/0-3/")[-1].split("/")[0]
    return uid

deal_id = get_contact_ID_from_URL(deal_url)
print("Deal ID:", deal_id)

Deal ID: 15220482319


### Get contact details

In [21]:
def retrieve_object_details(
    token,
    object_id,
    object_type,
    properties=None,
):
    # Init
    data = []
    params = {
        "archived": "false"
    }
    
    # Requests
    if properties:
        params["properties"] = properties
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v3/objects/{object_type}/{object_id}"
    
    # Response
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        data = res.json().get("properties")
    else:
        print(res.text)
    return pd.DataFrame([data])

def create_activity_df(
    token,
    object_id,
    activity,
    properties_dict=None,
):
    # Init
    properties = [x for x in properties_dict]
    
    # List activities
    df = retrieve_object_details(
        token,
        object_id,
        activity,
        properties
    )
    if len(df) > 0:
        df = df[properties]
        
    if len(df) > 0:
        df = df.rename(columns=properties_dict)
        if 'activity_type' not in df:
            df.insert(loc=1, column="activity_type", value=activity.upper())
    
    return df.reset_index(drop=True)

def get_details(
    hs_access_token,
    hubspot_id,
    properties,
    associations
):
    # Init
    message = "DEAL:\n"
    df = pd.DataFrame()
    
    # Get object
    obj = hubspot.connect(hs_access_token).deals.get(
        hubspot_id,
        hs_properties=properties,
        hs_associations=associations
    )
    
    # Get properties
    obj_properties = obj.get("properties")
    for p in properties:
        message = f"{message}- {p}: {obj_properties.get(p)}\n"
    
    # Get contact associations
    obj_associations = obj.get("associations")
    for a in obj_associations:
        results = obj_associations.get(a).get("results")
        for r in results:
            if a == "communications":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_communication_channel_type": "activity_type",
                    "hs_body_preview": "activity_content"
                }
            elif a == "meetings":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_meeting_title": "activity_content"
                }
            else:
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_body_preview": "activity_content"
                }
            association_id = r.get("id")
            
            # Create activity df
            tmp_df = create_activity_df(
                hs_access_token,
                association_id,
                a,
                properties_dict
            )
            df = pd.concat([df, tmp_df])
    
    # Cleaning df
    if len(df) > 0:
        # Exclude empty or None value
        df = df[~(df["activity_content"].astype(str).isin(["None"]))]

        # Format date
        df["activity_date"] = pd.to_datetime(df["activity_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
        df = df.sort_values(by="activity_date", ascending=False).reset_index(drop=True)
            
    # Create activity message
    message = f"{message}\nACTIVITIES:\n"
    for row in df.itertuples():
        activity_date = row.activity_date
        activity_type = row.activity_type
        activity_content = row.activity_content.replace("\xa0\u200c", "")
        message = f"{message}-{activity_date}: {activity_type} - {activity_content}\n"
    return message, df.reset_index(drop=True)

brief, df_activity = get_details(
    hs_access_token,
    deal_id,
    deal_properties,
    associations
)
print(brief)

DEAL:
- hs_object_id: 15220482319
- dealname: Agicap X Icelex connexion Quadra
- amount: None

ACTIVITIES:
-2023-09-26 07:44:00: EMAILS - Hello Florent,  Je t'appelle dans la journée. Bien à toi,  Pierre
-2023-09-26 07:40:00: EMAILS - Hello Pierre, Je me permets de te relancer par rapport à nos derniers échanges par email et téléphone. Est-ce que tu as fait un retour au Groupe Ice? Est-ce que tu aurais d'autres clients/prospects à nous présenter? On peut s'appeler dans la journée si besoin, Cordialement, Florent
-2023-09-21 14:15:00: EMAILS - Salut Pierre, Bonne nouvelle! Nous avons identifié un partenaire pour prendre en charge l'intégration entre QUADRA & Agicap. Comme mentionné lors de notre conversation téléphonique, je t'envoie notre analyse pour réaliser la connexion entre QUADRA & Agicap. Nous envisageons d'utiliser un robot pour simuler les actions humaines et réaliser les exports automatiquement. Cela nécessitera deux phases distinctes : La mise en place, qui comprend : L'inst

## Output

### Save DataFrame to CSV

In [22]:
df_activity.to_csv(f"Deal_activity_{deal_id}.csv", index=False)

### JSON response
Response to be sent in your Naas Chat input bar.

In [23]:
naas.webhook.respond_json(
    {
        "status": "ok", 
        "message": message.replace("[URL]", deal_url).replace("[ACTIVITIES]", brief)
    }
)

Response Set as JSON, preview below: 

<IPython.core.display.JSON object>

### Push Webhook to production
Webhook URL will be include in command of your Naas Chat plugin.

In [24]:
webhook_url = naas.webhook.add(params={"inline": True})

👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete


### Create command to be used in your Naas Chat

In [25]:
commands = [
    {
        "name": "HubSpot_Get_deal_summary",
        "action": {
          "request_type": "POST",
          "url": webhook_url,
          "payload": {
              "deal_url": {
                  "type": "str",
                  "description": "HubSpot deal URL: https://app.hubspot.com/contacts/2474088/record/0-3/15220482319/",
                  "default": "https://app.hubspot.com/contacts/2474088/record/0-3/15220482319/"
              },
          }
        }
    }
]

### Generate plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [26]:
plugin_path = naas_chat_plugin.create_plugin(
    name=plugin_name,
    prompt=system_prompt,
    model='gpt-3.5-turbo-16k',
    temperature=1,
    commands=commands,
)

✅ System prompt tokens count OK: 241 (limit: 20% -> 3277)
💾 Plugin successfully saved. You can use it in your Naas Chat with: sales_agent_plugin.json


### Create naas asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the cell below.

In [27]:
plugin_url = naas.asset.add(plugin_path, params={"inline": True})

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


### Create new chat

In [12]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")

[Create New Chat](https://naas.ai/chat/use?plugin_url=https://public.naas.ai/ZmxvcmVudC00MG5hYXMtMkVhaQ==/asset/d7b89b94307ec625ced0ae69dd46cd06211cd2b29d8c1e476df4aea25391)

***NB: If you want to update the prompt plugin sent in your Naas Chat, you must use the command `/refresh`. This command will retrieve the last prompt sent as an asset in Naas Lab.***